In [1]:
import keras
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np


Using TensorFlow backend.


In [2]:
batch_size = 64
epochs = 500
latent_dim = 256
num_samples = 10000

data_path = 'D:\\NLP_data\\tam.txt'

In [3]:
# Vectorizing the data
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

with open(data_path, 'r', encoding = 'utf-8') as f:
    lines = f.read().split('\n')
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text, _ = line.split('\t')
    #We use tab as the start sequence character
    # for the targets, and \n as end sequence character
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [4]:
#Printing the input characters
input_characters

{' ',
 '!',
 "'",
 ',',
 '.',
 '0',
 '2',
 '?',
 'A',
 'B',
 'C',
 'D',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'P',
 'R',
 'S',
 'T',
 'W',
 'Y',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z'}

In [5]:
# Printing the target characters
target_characters

{'\t',
 '\n',
 ' ',
 '!',
 '(',
 ')',
 ',',
 '.',
 '0',
 '2',
 '?',
 'C',
 'D',
 'அ',
 'ஆ',
 'இ',
 'உ',
 'ஊ',
 'எ',
 'ஏ',
 'ஒ',
 'ஓ',
 'க',
 'ங',
 'ச',
 'ஜ',
 'ஞ',
 'ட',
 'ண',
 'த',
 'ந',
 'ன',
 'ப',
 'ம',
 'ய',
 'ர',
 'ற',
 'ல',
 'ள',
 'ழ',
 'வ',
 'ஷ',
 'ஸ',
 'ா',
 'ி',
 'ீ',
 'ு',
 'ூ',
 'ெ',
 'ே',
 'ை',
 'ொ',
 'ோ',
 '்'}

In [6]:
# Printing the input texts
input_texts

['I slept.',
 'Calm down.',
 "I'll walk.",
 'Who is he?',
 'Who knows?',
 'She smiled.',
 'Talk to me!',
 'Who is she?',
 'Go to sleep.',
 'It may rain.',
 'She bit him.',
 'She hit him.',
 'She is kind.',
 'She is eight.',
 'Where are we?',
 'Keep in touch!',
 'See you again.',
 'Give it to her.',
 'I ate too much.',
 "I'll see to it.",
 "It's up to you.",
 'Leave it to me.',
 'Listen to this!',
 "That's the way.",
 'Come and see me.',
 "Don't lie to me.",
 'He began to run.',
 'He just arrived.',
 'He likes to run.',
 'How is your dad?',
 'I want to sleep.',
 "I'm able to run.",
 'Raise your hand.',
 'What did he say?',
 'When can we eat?',
 'Come and help us.',
 'He is still here.',
 'I have to go now.',
 'I know that much.',
 'I made a mistake.',
 'I walk to school.',
 "That's our house.",
 'Those are my CDs.',
 'Walk ahead of me.',
 "We'll follow you.",
 'Beware of the dog!',
 'He came back soon.',
 'He has three sons.',
 'I know how to ski.',
 'I know what to do.',
 "I'm kind of 

In [7]:
# Printing the target texts
target_texts

['\tநான் தூங்கினேன்.\n',
 '\tஅமைதியாக இருங்கள்\n',
 '\tநான் நடப்பேன்.\n',
 '\tஅவன் யார்?\n',
 '\tயாருக்குத் தெரியும்?\n',
 '\tஅவள் சிரித்தாள்\n',
 '\tஎன்னிடம் பேசு\n',
 '\tஅவள் யார்?\n',
 '\tபோய் தூங்கு\n',
 '\tமழை பெய்யலாம்\n',
 '\tஅவள் அவனைக் கடித்தாள்\n',
 '\tஅவள் அவனைக் அடித்தாள்\n',
 '\tஅவள் அன்பானவள்\n',
 '\tஅவளுக்கு எட்டு வயது\n',
 '\tநாம் எங்கே இருக்கிறோம்?\n',
 '\tதொடர்பில் இரு\n',
 '\tமறுபடியும் சந்திப்போம்\n',
 '\tஅவளிடம் கொடு\n',
 '\tநான் நிறைய சாப்பிட்டேன்\n',
 '\tஅதை நான் பார்க்கிறேன்\n',
 '\tஉன் கையில்தான் இருக்கிறது\n',
 '\tஎன்னிடம் விட்டுவிடு\n',
 '\tஇதைக் கேள்\n',
 '\tஅந்த பக்கம்தான் வழி\n',
 '\tஎன்னை வந்து பார்\n',
 '\tஎன்னிடம் பொய் சொல்லாதே\n',
 '\tஅவன் ஓட ஆரம்பித்தான்\n',
 '\tஅவன் இப்பொழுதுதான் வந்தான்\n',
 '\tஅவன் ஓட விருப்பப் படுகிறான்\n',
 '\tதங்களுடைய தந்தையார் எப்படி இருக்கிறார்கள்?\n',
 '\tநான் தூங்க விரும்புகிறேன்\n',
 '\tஎன்னால் ஓட முடிகிறது\n',
 '\tகையைத் தூக்கு\n',
 '\tஅவன் என்ன சொன்னான்?\n',
 '\tஎப்பொழுது நம்மால் சாப்பிட முடியும்\n',
 '\tவா எங்களுக்கு உத

In [8]:
len(target_characters)

54

In [9]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

In [10]:
print('Number of Samples:', len(input_texts))
print('No. of unique input tokens:', num_encoder_tokens)
print('No. of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequene length for outputs:', max_decoder_seq_length)

Number of Samples: 203
No. of unique input tokens: 53
No. of unique output tokens: 54
Max sequence length for inputs: 94
Max sequene length for outputs: 111


In [11]:
#Assigning token to each and every character

input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

In [12]:
input_token_index

{' ': 0,
 '!': 1,
 "'": 2,
 ',': 3,
 '.': 4,
 '0': 5,
 '2': 6,
 '?': 7,
 'A': 8,
 'B': 9,
 'C': 10,
 'D': 11,
 'F': 12,
 'G': 13,
 'H': 14,
 'I': 15,
 'J': 16,
 'K': 17,
 'L': 18,
 'M': 19,
 'N': 20,
 'P': 21,
 'R': 22,
 'S': 23,
 'T': 24,
 'W': 25,
 'Y': 26,
 'a': 27,
 'b': 28,
 'c': 29,
 'd': 30,
 'e': 31,
 'f': 32,
 'g': 33,
 'h': 34,
 'i': 35,
 'j': 36,
 'k': 37,
 'l': 38,
 'm': 39,
 'n': 40,
 'o': 41,
 'p': 42,
 'q': 43,
 'r': 44,
 's': 45,
 't': 46,
 'u': 47,
 'v': 48,
 'w': 49,
 'x': 50,
 'y': 51,
 'z': 52}

In [13]:
target_token_index

{'\t': 0,
 '\n': 1,
 ' ': 2,
 '!': 3,
 '(': 4,
 ')': 5,
 ',': 6,
 '.': 7,
 '0': 8,
 '2': 9,
 '?': 10,
 'C': 11,
 'D': 12,
 'அ': 13,
 'ஆ': 14,
 'இ': 15,
 'உ': 16,
 'ஊ': 17,
 'எ': 18,
 'ஏ': 19,
 'ஒ': 20,
 'ஓ': 21,
 'க': 22,
 'ங': 23,
 'ச': 24,
 'ஜ': 25,
 'ஞ': 26,
 'ட': 27,
 'ண': 28,
 'த': 29,
 'ந': 30,
 'ன': 31,
 'ப': 32,
 'ம': 33,
 'ய': 34,
 'ர': 35,
 'ற': 36,
 'ல': 37,
 'ள': 38,
 'ழ': 39,
 'வ': 40,
 'ஷ': 41,
 'ஸ': 42,
 'ா': 43,
 'ி': 44,
 'ீ': 45,
 'ு': 46,
 'ூ': 47,
 'ெ': 48,
 'ே': 49,
 'ை': 50,
 'ொ': 51,
 'ோ': 52,
 '்': 53}

In [14]:
# One hot representation
encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype = 'float32')

decoder_input_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype = 'float32')

decoder_target_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype = 'float32')

In [15]:
# This piece of code does One-Hot representation
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    encoder_input_data[i, t + 1:, input_token_index[' ']] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            #decoder_target_data will be ahead of one timestep
            #and will not include the start character
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.
    decoder_input_data[i, t + 1:, target_token_index[' ']] = 1.
    decoder_target_data[i, t:, target_token_index[' ']] = 1.

In [16]:
encoder_input_data[0].shape

(94, 53)

In [17]:
# Defining an input sentence and processing it

encoder_inputs = Input(shape = (None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state = True)
encoder_outpus4ts, state_h, state_c = encoder(encoder_inputs)

# We discard 'encoder_outputs' and keep the states
encoder_states = [state_h, state_c]

In [18]:
# Setting up the decoder using 'encoder_states' as initial state
decoder_inputs = Input(shape = (None, num_decoder_tokens))

# We set up our decoder to return full output sequences and to return
# internal states as well. We don't use the return states in the
# training model, but we will use them in inference.

decoder_lstm = LSTM(latent_dim, return_sequences = True, return_state = True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, 
                                     initial_state = encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation = 'softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [19]:
# Defining the model that will turn 'encoder_input_data' &
# 'decoder_input_data' into 'decoder_target_data'

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(optimizer = 'rmsprop', loss = 'categorical_crossentropy',
              metrics = ['accuracy'])

model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size = batch_size, epochs = epochs,
          validation_split = 0.2)

Train on 162 samples, validate on 41 samples
Epoch 1/500
162/162 [==============================] - 2s 13ms/step - loss: 3.1814 - accuracy: 0.4662 - val_loss: 2.7874 - val_accuracy: 0.5865
Epoch 2/500
162/162 [==============================] - 1s 4ms/step - loss: 1.7530 - accuracy: 0.7533 - val_loss: 2.2270 - val_accuracy: 0.5922
Epoch 3/500
162/162 [==============================] - 1s 4ms/step - loss: 1.1429 - accuracy: 0.7612 - val_loss: 2.0514 - val_accuracy: 0.5906
Epoch 4/500
162/162 [==============================] - 1s 4ms/step - loss: 1.1178 - accuracy: 0.7602 - val_loss: 2.3320 - val_accuracy: 0.5895
Epoch 5/500
162/162 [==============================] - 1s 4ms/step - loss: 1.1837 - accuracy: 0.7608 - val_loss: 1.9988 - val_accuracy: 0.5926
Epoch 6/500
162/162 [==============================] - 1s 4ms/step - loss: 1.0899 - accuracy: 0.7609 - val_loss: 2.1516 - val_accuracy: 0.5920
Epoch 7/500
162/162 [==============================] - 1s 4ms/step - loss: 1.1262 - accuracy: 0.

162/162 [==============================] - 1s 4ms/step - loss: 0.7831 - accuracy: 0.7833 - val_loss: 2.3409 - val_accuracy: 0.6010
Epoch 58/500
162/162 [==============================] - 1s 4ms/step - loss: 1.0390 - accuracy: 0.7892 - val_loss: 1.3749 - val_accuracy: 0.6423
Epoch 59/500
162/162 [==============================] - 1s 4ms/step - loss: 0.7479 - accuracy: 0.8028 - val_loss: 1.3661 - val_accuracy: 0.6447
Epoch 60/500
162/162 [==============================] - 1s 4ms/step - loss: 0.7370 - accuracy: 0.8035 - val_loss: 1.3631 - val_accuracy: 0.6449
Epoch 61/500
162/162 [==============================] - 1s 4ms/step - loss: 0.7314 - accuracy: 0.8026 - val_loss: 1.3589 - val_accuracy: 0.6429
Epoch 62/500
162/162 [==============================] - 1s 4ms/step - loss: 0.7300 - accuracy: 0.8024 - val_loss: 1.3677 - val_accuracy: 0.6313
Epoch 63/500
162/162 [==============================] - 1s 4ms/step - loss: 0.7307 - accuracy: 0.8002 - val_loss: 1.3547 - val_accuracy: 0.6434
Epoch

Epoch 114/500
162/162 [==============================] - 1s 4ms/step - loss: 0.4912 - accuracy: 0.8615 - val_loss: 1.0749 - val_accuracy: 0.7049
Epoch 115/500
162/162 [==============================] - 1s 5ms/step - loss: 0.4856 - accuracy: 0.8635 - val_loss: 1.0917 - val_accuracy: 0.6987
Epoch 116/500
162/162 [==============================] - 1s 8ms/step - loss: 0.4882 - accuracy: 0.8600 - val_loss: 1.0976 - val_accuracy: 0.7001
Epoch 117/500
162/162 [==============================] - 1s 3ms/step - loss: 0.4871 - accuracy: 0.8618 - val_loss: 1.1085 - val_accuracy: 0.6963
Epoch 118/500
162/162 [==============================] - 1s 4ms/step - loss: 0.4811 - accuracy: 0.8621 - val_loss: 1.0686 - val_accuracy: 0.6972
Epoch 119/500
162/162 [==============================] - 1s 4ms/step - loss: 0.4726 - accuracy: 0.8645 - val_loss: 1.0578 - val_accuracy: 0.7082
Epoch 120/500
162/162 [==============================] - 1s 4ms/step - loss: 0.4719 - accuracy: 0.8651 - val_loss: 1.0884 - val_ac

162/162 [==============================] - 1s 7ms/step - loss: 0.3296 - accuracy: 0.9043 - val_loss: 1.1285 - val_accuracy: 0.7091
Epoch 171/500
162/162 [==============================] - 1s 6ms/step - loss: 0.3241 - accuracy: 0.9090 - val_loss: 1.1727 - val_accuracy: 0.7023
Epoch 172/500
162/162 [==============================] - 1s 3ms/step - loss: 0.3195 - accuracy: 0.9087 - val_loss: 1.1586 - val_accuracy: 0.6990
Epoch 173/500
162/162 [==============================] - 1s 5ms/step - loss: 0.3117 - accuracy: 0.9125 - val_loss: 1.1689 - val_accuracy: 0.7003
Epoch 174/500
162/162 [==============================] - 1s 5ms/step - loss: 0.3141 - accuracy: 0.9104 - val_loss: 1.1686 - val_accuracy: 0.7080
Epoch 175/500
162/162 [==============================] - 1s 3ms/step - loss: 0.3154 - accuracy: 0.9110 - val_loss: 1.1630 - val_accuracy: 0.6990
Epoch 176/500
162/162 [==============================] - 1s 4ms/step - loss: 0.3100 - accuracy: 0.9132 - val_loss: 1.1590 - val_accuracy: 0.7047

162/162 [==============================] - 1s 6ms/step - loss: 0.1838 - accuracy: 0.9520 - val_loss: 1.3804 - val_accuracy: 0.6946
Epoch 227/500
162/162 [==============================] - 1s 4ms/step - loss: 0.1792 - accuracy: 0.9526 - val_loss: 1.3802 - val_accuracy: 0.6966
Epoch 228/500
162/162 [==============================] - 1s 4ms/step - loss: 0.1729 - accuracy: 0.9559 - val_loss: 1.3609 - val_accuracy: 0.6950
Epoch 229/500
162/162 [==============================] - 1s 3ms/step - loss: 0.1797 - accuracy: 0.9525 - val_loss: 1.4105 - val_accuracy: 0.6908
Epoch 230/500
162/162 [==============================] - 1s 4ms/step - loss: 0.1880 - accuracy: 0.9491 - val_loss: 1.3681 - val_accuracy: 0.6941
Epoch 231/500
162/162 [==============================] - 1s 4ms/step - loss: 0.1730 - accuracy: 0.9552 - val_loss: 1.4021 - val_accuracy: 0.6941
Epoch 232/500
162/162 [==============================] - 1s 4ms/step - loss: 0.1671 - accuracy: 0.9565 - val_loss: 1.3833 - val_accuracy: 0.6919

162/162 [==============================] - 1s 5ms/step - loss: 0.0992 - accuracy: 0.9735 - val_loss: 1.6233 - val_accuracy: 0.6845
Epoch 283/500
162/162 [==============================] - 1s 5ms/step - loss: 0.1064 - accuracy: 0.9725 - val_loss: 1.6599 - val_accuracy: 0.6856
Epoch 284/500
162/162 [==============================] - 1s 3ms/step - loss: 0.0997 - accuracy: 0.9748 - val_loss: 1.6346 - val_accuracy: 0.6790
Epoch 285/500
162/162 [==============================] - 1s 3ms/step - loss: 0.0979 - accuracy: 0.9746 - val_loss: 1.6480 - val_accuracy: 0.6904
Epoch 286/500
162/162 [==============================] - 1s 6ms/step - loss: 0.1092 - accuracy: 0.9716 - val_loss: 1.6342 - val_accuracy: 0.6831
Epoch 287/500
162/162 [==============================] - 1s 6ms/step - loss: 0.1053 - accuracy: 0.9730 - val_loss: 1.6507 - val_accuracy: 0.6873
Epoch 288/500
162/162 [==============================] - 1s 3ms/step - loss: 0.0937 - accuracy: 0.9756 - val_loss: 1.6176 - val_accuracy: 0.6864

162/162 [==============================] - 1s 3ms/step - loss: 0.0712 - accuracy: 0.9790 - val_loss: 1.8279 - val_accuracy: 0.6825
Epoch 339/500
162/162 [==============================] - 1s 6ms/step - loss: 0.0684 - accuracy: 0.9795 - val_loss: 1.8295 - val_accuracy: 0.6853
Epoch 340/500
162/162 [==============================] - 1s 7ms/step - loss: 0.0672 - accuracy: 0.9801 - val_loss: 1.8422 - val_accuracy: 0.6823
Epoch 341/500
162/162 [==============================] - 1s 5ms/step - loss: 0.0672 - accuracy: 0.9801 - val_loss: 1.8092 - val_accuracy: 0.6884
Epoch 342/500
162/162 [==============================] - 1s 9ms/step - loss: 0.0675 - accuracy: 0.9795 - val_loss: 1.8386 - val_accuracy: 0.6849
Epoch 343/500
162/162 [==============================] - 1s 4ms/step - loss: 0.0728 - accuracy: 0.9784 - val_loss: 1.8332 - val_accuracy: 0.6823
Epoch 344/500
162/162 [==============================] - 1s 5ms/step - loss: 0.0665 - accuracy: 0.9808 - val_loss: 1.8795 - val_accuracy: 0.6829

162/162 [==============================] - 1s 4ms/step - loss: 0.0590 - accuracy: 0.9808 - val_loss: 1.9640 - val_accuracy: 0.6836
Epoch 395/500
162/162 [==============================] - 1s 8ms/step - loss: 0.0563 - accuracy: 0.9809 - val_loss: 1.9614 - val_accuracy: 0.6840
Epoch 396/500
162/162 [==============================] - 1s 4ms/step - loss: 0.0576 - accuracy: 0.9813 - val_loss: 1.9604 - val_accuracy: 0.6820
Epoch 397/500
162/162 [==============================] - 1s 7ms/step - loss: 0.0570 - accuracy: 0.9814 - val_loss: 1.9770 - val_accuracy: 0.6845
Epoch 398/500
162/162 [==============================] - 1s 6ms/step - loss: 0.0573 - accuracy: 0.9811 - val_loss: 1.9685 - val_accuracy: 0.6836
Epoch 399/500
162/162 [==============================] - 1s 4ms/step - loss: 0.0574 - accuracy: 0.9806 - val_loss: 1.9961 - val_accuracy: 0.6823
Epoch 400/500
162/162 [==============================] - 1s 4ms/step - loss: 0.0584 - accuracy: 0.9808 - val_loss: 1.9529 - val_accuracy: 0.6853

162/162 [==============================] - 1s 5ms/step - loss: 0.0522 - accuracy: 0.9824 - val_loss: 2.0431 - val_accuracy: 0.6873
Epoch 451/500
162/162 [==============================] - 1s 7ms/step - loss: 0.0521 - accuracy: 0.9823 - val_loss: 2.0431 - val_accuracy: 0.6858
Epoch 452/500
162/162 [==============================] - 1s 4ms/step - loss: 0.0539 - accuracy: 0.9819 - val_loss: 2.0437 - val_accuracy: 0.6864
Epoch 453/500
162/162 [==============================] - 1s 6ms/step - loss: 0.0527 - accuracy: 0.9823 - val_loss: 2.0828 - val_accuracy: 0.6882
Epoch 454/500
162/162 [==============================] - 1s 7ms/step - loss: 0.0525 - accuracy: 0.9822 - val_loss: 2.0719 - val_accuracy: 0.6849
Epoch 455/500
162/162 [==============================] - 1s 3ms/step - loss: 0.0543 - accuracy: 0.9814 - val_loss: 2.0538 - val_accuracy: 0.6856
Epoch 456/500
162/162 [==============================] - 1s 4ms/step - loss: 0.0513 - accuracy: 0.9824 - val_loss: 2.0496 - val_accuracy: 0.6860

In [20]:
# Next: inference mode (sampling)
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
#  and a 'start of sequence' token as target
# output will be the next target token
# 3) repeat with the current target token and current states

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape = (latent_dim,))
decoder_state_input_c = Input(shape = (latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, 
                                                 initial_state = decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model([decoder_inputs] + decoder_states_inputs, 
                      [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to something readable
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

def decode_sequence(input_seq):
    # Encode input as state vectors
    states_value = encoder_model.predict(input_seq)
    
    # Generate empty target sequence of length 1
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    
    # populate the first character of target sequence with start character
    target_seq[0, 0, target_token_index['\t']] = 1.
    
    # sampling loop for a batch of sequernces
    # (to simplify, here we assume a batch size of 1)
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        
        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char
        
        # Exit condition: either hit max length or find stop character
        if (sampled_char == '\n' or 
            len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True
            
        # Update the target sequence (of length 1)
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.
        
        # Update states
        states_value = [h, c]
    
    return decoded_sentence


for seq_index in range(100):
    # Take one sequence (part of the training set) for trying out decoding
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: I slept.
Decoded sentence: நான் தூங்க விரும்புகிறேன்

-
Input sentence: Calm down.
Decoded sentence: அவன் சீக்கிரம் திரும்பி வந்தான்

-
Input sentence: I'll walk.
Decoded sentence: நான் தூங்க விரும்புகிறேன்

-
Input sentence: Who is he?
Decoded sentence: அவன் சீக்கிரம் திரும்பி வந்தான்

-
Input sentence: Who knows?
Decoded sentence: அவன் சீக்கிரம் திரும்பி வந்தான்

-
Input sentence: She smiled.
Decoded sentence: அவள் அவனைக் கடித்தாள்

-
Input sentence: Talk to me!
Decoded sentence: நான் தூங்க விரும்புகிறேன்

-
Input sentence: Who is she?
Decoded sentence: அவன் சீக்கிரம் திரும்பி வந்தான்

-
Input sentence: Go to sleep.
Decoded sentence: அவள் அவனைக் கடித்தாள்

-
Input sentence: It may rain.
Decoded sentence: நான் தூங்க விரும்புகிறேன்

-
Input sentence: She bit him.
Decoded sentence: அவள் அவனைக் கடித்தாள்

-
Input sentence: She hit him.
Decoded sentence: அவள் அவனைக் கடித்தாள்

-
Input sentence: She is kind.
Decoded sentence: அவள் அவனைக் கடித்தாள்

-
Input sentence: She i